<a href="https://colab.research.google.com/github/FJPO/citation_predictor/blob/main/Internship_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam



In [11]:
data = pd.read_csv('predict.csv')

yScaler = preprocessing.MinMaxScaler()
y_train = yScaler.fit_transform(np.array(data['c5']).reshape(-1,1))
x_train = preprocessing.MinMaxScaler().fit_transform(np.array(data['recency']).reshape(-1, 1))
x_train = np.c_[x_train,
                preprocessing.MinMaxScaler().fit_transform(np.array(data['topic_rank']).reshape(-1, 1)),
                preprocessing.MinMaxScaler().fit_transform(np.array(data['diversity']).reshape(-1, 1)),
                preprocessing.MinMaxScaler().fit_transform(np.array(data['authors_mean_rank']).reshape(-1, 1)),
                preprocessing.MinMaxScaler().fit_transform(np.array(data['authors_mean_hindex']).reshape(-1, 1)),
                preprocessing.MinMaxScaler().fit_transform(np.array(data['authors_mean_sociality']).reshape(-1, 1)),
                preprocessing.MinMaxScaler().fit_transform(np.array(data['title_len']).reshape(-1, 1)),
                preprocessing.MinMaxScaler().fit_transform(np.array(data['abstract_len']).reshape(-1, 1)),
                preprocessing.MinMaxScaler().fit_transform(np.array(data['authors_mean_hindex']).reshape(-1, 1)),
                preprocessing.MinMaxScaler().fit_transform(np.array(data['journal_rank']).reshape(-1, 1)),
                preprocessing.MinMaxScaler().fit_transform(np.array(data['log_authors_mean_sociality']).reshape(-1, 1)),

                ]
# preprocessing.MinMaxScaler(np.array(data['topic_rank']).reshape(-1, 1))
x_train = x_train[~np.isnan(x_train).any(axis = 1)]


In [12]:
model = Sequential()
model.add(Dense(150, input_dim = x_train.shape[1], activation = 'relu'))
model.add(Dense(150, activation = 'relu'))
model.add(Dense(1))
model.compile(optimizer = Adam(lr = 0.001), loss = 'mse')

model.fit(x_train, y_train, 16, 10, verbose=1, validation_split=0.2)
model.predict(x_train[:5])

Epoch 1/10
22/22 [==============================] - 1s 10ms/step - loss: 0.0094 - val_loss: 0.0162
Epoch 2/10
22/22 [==============================] - 0s 4ms/step - loss: 0.0056 - val_loss: 0.0176
Epoch 3/10
22/22 [==============================] - 0s 4ms/step - loss: 0.0049 - val_loss: 0.0172
Epoch 4/10
22/22 [==============================] - 0s 3ms/step - loss: 0.0036 - val_loss: 0.0171
Epoch 5/10
22/22 [==============================] - 0s 4ms/step - loss: 0.0071 - val_loss: 0.0179
Epoch 6/10
22/22 [==============================] - 0s 3ms/step - loss: 0.0068 - val_loss: 0.0200
Epoch 7/10
22/22 [==============================] - 0s 4ms/step - loss: 0.0043 - val_loss: 0.0177
Epoch 8/10
22/22 [==============================] - 0s 4ms/step - loss: 0.0061 - val_loss: 0.0183
Epoch 9/10
22/22 [==============================] - 0s 4ms/step - loss: 0.0040 - val_loss: 0.0166
Epoch 10/10
22/22 [==============================] - 0s 4ms/step - loss: 0.0058 - val_loss: 0.0230


array([[0.03022282],
       [0.05188459],
       [0.01814761],
       [0.06640895],
       [0.04429096]], dtype=float32)

In [13]:
y_test_from_model = yScaler.inverse_transform(model.predict(x_train))
y_actual = yScaler.inverse_transform(y_train)
for i in range(len(y_test_from_model)):
  print(int(y_test_from_model[i]) if (y_test_from_model[i] > 0) else 0, '\t', int(y_actual[i][0]))
diff = y_test_from_model - y_actual
err = np.mean(abs(diff[~np.isnan(diff)]))
print(err)

4 	 1
7 	 32
2 	 5
10 	 7
6 	 7
4 	 5
0 	 1
3 	 5
3 	 8
5 	 7
6 	 4
5 	 9
4 	 0
6 	 2
12 	 29
4 	 9
6 	 15
4 	 5
4 	 9
14 	 2
7 	 5
0 	 2
8 	 46
15 	 48
6 	 2
10 	 1
0 	 1
7 	 8
7 	 1
4 	 5
0 	 0
3 	 1
0 	 1
5 	 1
7 	 8
7 	 4
3 	 2
3 	 2
6 	 7
3 	 1
9 	 27
5 	 5
1 	 9
13 	 9
6 	 5
8 	 10
5 	 14
4 	 15
8 	 0
1 	 5
0 	 11
4 	 8
6 	 4
9 	 8
4 	 0
5 	 1
6 	 5
8 	 18
4 	 8
3 	 1
3 	 2
3 	 0
4 	 7
2 	 4
8 	 14
8 	 4
8 	 19
3 	 11
8 	 2
6 	 2
7 	 0
5 	 0
4 	 13
8 	 9
6 	 22
0 	 0
0 	 1
8 	 7
8 	 7
7 	 19
5 	 4
2 	 5
10 	 4
3 	 2
0 	 1
0 	 7
5 	 2
1 	 2
6 	 16
5 	 2
0 	 5
6 	 14
8 	 5
5 	 0
1 	 7
4 	 5
6 	 2
2 	 2
4 	 0
4 	 13
10 	 2
6 	 11
6 	 2
3 	 1
7 	 0
4 	 1
6 	 11
8 	 2
8 	 5
10 	 13
1 	 2
6 	 7
6 	 46
6 	 23
7 	 11
5 	 2
4 	 39
4 	 2
12 	 29
4 	 2
3 	 1
9 	 2
7 	 1
5 	 14
5 	 9
2 	 2
3 	 1
8 	 13
7 	 4
4 	 2
8 	 1
8 	 4
6 	 1
6 	 0
7 	 1
0 	 0
2 	 1
5 	 5
5 	 2
2 	 7
8 	 2
1 	 0
6 	 7
5 	 1
0 	 0
3 	 2
6 	 17
5 	 1
7 	 5
6 	 24
6 	 2
8 	 11
4 	 2
0 	 4
16 	 19
6 	 16
4 	 5
0 	 0
7 	 10

ValueError: ignored

0.0118 on recency only
0.0115 on recency and topick_rank